# Perspektiva - měření vzdálenosti v obraze
Cvičení je velmi krátké a jednoduché. Nicméně, je prerekvizitou k úspěšnému absolvování druhého domácího úkolu. 

S tématem měření vzdáleností v obraze jsme se setkali již ve 3. cvičení. Připomeňme si však, jakým způsobem jsme problém řešili. Využívali jsme znalosti poměru px/mm. Tento způsob měl jednu velkou nevýhodu, neboť když se nebude obrazová rovina nacházet rovnoběžně se snímanou rovinou, poměr px/mm nebude dávat smysl a budeme dostávat chybné výsledky.

Tento problém vyřešíme znalostí perspektivy, konkrétně aplikací 2D perspektivní transformace (viz přednáška [Geometrické transformace obrazu](../../../lectures/files/bi-svz-06-metody-predzpracovani-obrazu-1.pdf) - homography). Představme si nyní obrázek, na kterém chceme zjistit skutečnou vzdálenost dvou bodů označených úsečkou **`d`**  v milimetrech.

![](images/pattern_measuring.jpg)

Na první pohled je jasné, že počítat poměr px/mm v tomto obraze je nesmyslné. Obraz je perspektivně zkreslený - kruhy se změnily na elipsy a obdélníky se staly lichoběžníky. Se znalostí toho, jak vypadaly původní tvary v obraze, a jaké byly jejich původní rozměry, jsme schopni papír se vzory geometricky transformovat do tvaru "kolmého k obrazové rovině". Navíc, nejenže známe velikosti referenčního oranžového obdélníku, ale známe také velikosti klasického A4 papíru (210 x 297 mm). Informace o rozměrech a tvaru papíru nám stačí k tomu, vypočítat transformační matici **`H`**, která každý pixel dokáže převést do nového "narovnaného" obrazu.

![](images/pattern_measuring_warped.jpg)

Pokud si v rámci této transformace ještě navíc zvolíme chytré jednotky. Výsledné vzdálenosti v pixelech budou odpovídat přímo vzdálenostem v mm nebo v čemkoliv jiném.

### Import knihoven a konfigurace

In [1]:
%run ../svz.ipynb

### Pomocné funkce
Z následujících funkcí je potřeba vybírat ty vhodné pro splnění úkolu.

Seznam funkcí pro přehlednost:
- [`show_images(...)`](../svz.ipynb#show_functions)

---

### Úkol

**1) Získejte a zobrazte si snímek fyzicky natočeného vzoru snímaného pod úhlem cca 30°.**

Nezapomínejte, že všechny kraje papíru musí být viditelné.

In [2]:
img = load_image('./images/good_img.bmp')

In [52]:
show_images(img)

x = 446, y = 609
x = 1321, y = 422


**2) Vytvořte si dva seznamy korenspondečních bodů, které budou využity pro výpočet transformační matice H**

V prvním seznamu musí být obrazové body. K jejich získáni využijte implementovaný callback ve funkci `show_images`, který po kliknutí do obrazu, vypíše souřadnice.
Druhý seznam musí obsahovat cílové souřadnice bodů, do kterých chceme vybrané body z prvního seznamu transformovat. 

Pozor, záleží na pořadí bodů a obě struktury musí být `np.array([..., ..., ..., ...])`.

In [43]:
image_pts = np.array([(132,91), (60, 987), (1904, 884), (1557, 90)]) # np.array of tuples
world_pts = np.array([(0,0), (0, 210), (297, 210), (297, 0)])
# (210 x 297 mm)

**3) Pomocí funkce [`cv2.findHomography`](https://docs.opencv.org/3.4.1/d9/d0c/group__calib3d.html#ga4abc2ece9fab9398f2e560d53c8c9780) získejte transformační matici `H`. Matici vypiště, ať si připomenete její tvar.**

In [44]:
import cv2
H, mask = cv2.findHomography(image_pts, world_pts)
print(H)
print(mask)

[[0.195 0.016 -27.142]
 [0.000 0.318 -29.004]
 [-0.000 0.000 1.000]]
[[1]
 [1]
 [1]
 [1]]


**4) Využijte [`cv2.warpPerspective`](https://docs.opencv.org/3.4.1/da/d54/group__imgproc__transform.html#gaf73673a7e8e18ec6963e3774e6a94b87), která přijímá transformační matici `H`, k narovnání zdrojového obrazu. Výsledek zobrazte.** 

In [45]:
img.shape

(1080, 1920, 3)

In [50]:
warped_img = cv2.warpPerspective(img, H, (297, 210))

In [51]:
show_images(warped_img)

**5) Ve vašem nasnímaném obraze získejte souřadnice hraničnich bodů úsečky `d` a uložte je do definovaných proměných.** 

Nelekejte se zběsilého předepsaného formátu, OpenCV bohužel občas požaduje zvláštní struktury.

In [53]:
p1 = 446, 609
p2 = 1321, 422

In [54]:
pt_src = np.array([[p1]], dtype='float32') # tuple
pt_dst = np.array([[p2]], dtype='float32') # tuple

**6) Využijte [`cv2.perspectiveTransform`](https://docs.opencv.org/3.4.1/d2/de8/group__core__array.html#gad327659ac03e5fd6894b90025e6900a7) k transformaci zdrojových bodů do nového prostoru a vypočítejte reálnou vzdálenost, kterou vypište.**

array([[[57.964, 138.037]]], dtype=float32)

In [64]:

pt_t_src = cv2.perspectiveTransform(pt_src, H)
pt_t_dst = cv2.perspectiveTransform(pt_dst, H)

dist = np.linalg.norm(pt_t_src - pt_t_dst)
print(f'D = {dist / 10:.02f} cm')

D = 16.78 cm
